In [ ]:
import torch
import os

# Forzar a que use la GPU de NVIDIA si hay dudas
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

print(f"¿Cuda disponible?: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Entrenando en: {torch.cuda.get_device_name(0)}")
else:
    # Si sale False, dinos qué sale aquí:
    print(f"Versión detectada: {torch.__version__}")

In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="klmVEuTiYxKVkKRwtyGF")
project = rf.workspace("samuel-gonzalez-duran-l6jsa").project("fashion-detector-n3bi3")
version = project.version(3)
dataset = version.download("yolov11")
                

In [ ]:
import ultralytics
from ultralytics import YOLO
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

model = YOLO("yolo11n.pt")
results = model.train(
    data="C:/Users/andsa/Documents/202526ULPGC/VC/trabajofinal/ForecastWear/Backend/Fashion-Detector-3/data.yaml",
    epochs = 50,
    imgsz=640,
    device=0,
    batch=4,    
    workers=4,   
    plots=True,
     )

In [ ]:
import ultralytics
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

def detect_person(image):
    model = YOLO('yolo11n.pt') 
    imagen = cv2.imread(image)
    results = model(imagen)
    boxes = results[0].boxes.xyxy.cpu().numpy()  
    scores = results[0].boxes.conf.cpu().numpy()  
    class_ids = results[0].boxes.cls.cpu().numpy()  
    class_names = model.names 
    crops_person = []
    for box, score, class_id in zip(boxes, scores, class_ids):
        class_id = int(class_id)
        if class_id == 0 and score >= 0.5: 
            x1, y1, x2, y2 = map(int, box)
            crops_person.append([x1, y1, x2, y2])
            label = f"{class_names[int(class_id)]}: {score:.2f}"
            cv2.rectangle(imagen, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(imagen, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)    
    if len(crops_person) == 0:
        print("No se detectó ninguna persona en la imagen.")
        return None
    else:
        person_croped = croped_person(crops_person[0],imagen)
        cv2.imwrite("funciona1.jpg",person_croped)
        person_croped_image = "funciona1.jpg"
        mujer_cholas = detect_clothes(person_croped_image)
        print(mujer_cholas[1])
        recorte_shoes = croped_person(mujer_cholas[1][0],mujer_cholas[0])
        cv2.imshow('Detectada',mujer_cholas[0])
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        

def croped_person(coordenates,imagen_rgb):
    return imagen_rgb[coordenates[1]:coordenates[3], coordenates[0]:coordenates[2]]

def detect_clothes(person):
    model = YOLO("C:/Users/andsa/Documents/202526ULPGC/VC/trabajofinal/ForecastWear/Backend/runs/detect/train2/weights/best.pt")
    image = cv2.imread(person)
    results = model(image)[0]
    class_names = model.names
    shoes_map = []
    for box in results.boxes:
        score = box.conf[0].item()
        class_id = int(box.cls[0].item())
        if score >= 0.5:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            if class_names[class_id] == "shoe":
                shoes_map.append([x1, y1, x2, y2])
            label = f"{class_names[class_id]}: {score:.2f}"
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    

    return [image,shoes_map]
   

if __name__ == "__main__":
    detect_person('cholas1.jpg')
    

In [1]:
import gradio as gr
from transformers import AutoImageProcessor, ViTForImageClassification
from PIL import Image
import torch
from transformers import  SegformerForSemanticSegmentation
import torch.nn as nn
import numpy as np

segment_processor = AutoImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")
segment_model = SegformerForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")
model_name = "google/vit-base-patch16-224"
processor = AutoImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name)
color_clothes = {
    0:  (0, 0, 0),        # Background - negro
    1:  (255, 105, 180),  # Hat - rosa
    3:  (0, 255, 255),    # Sunglasses - cian
    4:  (255, 0, 0),      # Upper-clothes - rojo
    5:  (0, 255, 0),      # Skirt - verde
    6:  (0, 0, 255),      # Pants - azul
    7:  (255, 255, 0),    # Dress - amarillo
    8:  (255, 165, 0),    # Belt - naranja
    9:  (255, 0, 255),    # Left-shoe - fucsia
    10: (128, 0, 128),    # Right-shoe - morado
    16: (0, 128, 128),    # Bag - verde azulado
    17: (128, 128, 128),  # Scarf - gris
}
classes_to_classify = {
    4: "Upper-clothes",
    5: "Skirt",
    6: "Pants",
    7: "Dress",
    9: "Left-shoe",
    10: "Right-shoe",
}


def colorize_mask(mask):
    h, w = mask.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)

    for cls, color in color_clothes.items():
        color_mask[mask == cls] = color

    return color_mask

def overlay_mask(image, mask, alpha=0.5):
    image_np = np.array(image)
    color_mask = colorize_mask(mask)

    overlay = image_np.copy()
    overlay[color_mask.sum(axis=2) > 0] = (
        (1 - alpha) * image_np[color_mask.sum(axis=2) > 0] +
        alpha * color_mask[color_mask.sum(axis=2) > 0]
    ).astype(np.uint8)

    return overlay


def segment_clothing(image):
    if image is None:
        return None

    image_pil = Image.fromarray(image).convert("RGB")

    inputs = segment_processor(images=image_pil, return_tensors="pt")
    with torch.no_grad():
        outputs = segment_model(**inputs)

    logits = outputs.logits
    upsampled_logits = nn.functional.interpolate(
        logits,
        size=image_pil.size[::-1],
        mode="bilinear",
        align_corners=False
    )

    mask = upsampled_logits.argmax(dim=1)[0].cpu().numpy()
    overlay = overlay_mask(image_pil, mask)

    return overlay

def extract_class(image, mask, class_id):
    """
    Devuelve una imagen donde solo la clase class_id aparece (el resto negro)
    """
    image_np = np.array(image)
    class_mask = (mask == class_id).astype(np.uint8)

    if class_mask.sum() == 0:
        return None  # No se detectó esta prenda

    class_mask_3ch = np.stack([class_mask]*3, axis=-1)
    segmented = image_np * class_mask_3ch
    return Image.fromarray(segmented)
def segment_and_classify_by_type(image):
    if image is None:
        return {}, None

    image_pil = Image.fromarray(image).convert("RGB")

    # 1️⃣ Segmentación
    inputs = segment_processor(images=image_pil, return_tensors="pt")
    with torch.no_grad():
        outputs = segment_model(**inputs)

    logits = outputs.logits
    upsampled_logits = nn.functional.interpolate(
        logits,
        size=image_pil.size[::-1],
        mode="bilinear",
        align_corners=False
    )
    mask = upsampled_logits.argmax(dim=1)[0].cpu().numpy()

    # Imagen overlay para mostrar
    overlay = overlay_mask(image_pil, mask).astype(np.uint8)

    # 2️⃣ Clasificación por clase
    results = {}
    for class_id, class_name in classes_to_classify.items():
        class_img = extract_class(image_pil, mask, class_id)
        if class_img is None:
            continue

        # Clasificación ViT
        inputs_vit = processor(images=class_img, return_tensors="pt")
        with torch.no_grad():
            outputs_vit = model(**inputs_vit)
            logits_vit = outputs_vit.logits
            probs = torch.nn.functional.softmax(logits_vit, dim=1).squeeze()
            top_prob, top_idx = torch.topk(probs, 1)
            predicted_label = model.config.id2label[top_idx.item()]
        
        results[class_name] = predicted_label

    return overlay, results


def classify_image(image):
    if image is None: return None
    image = Image.fromarray(image).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1).squeeze()
        top_probs, top_indices = torch.topk(probs, 5)
    
    return {model.config.id2label[idx.item()]: round(prob.item(), 3) for prob, idx in zip(top_probs, top_indices)}

css =""
with gr.Blocks(theme=gr.themes.Soft(),css=css) as demo:
    gr.Markdown("# 🌦️ Asesor del Clima")
    
    with gr.Column():
        input_img = gr.Image(type="numpy", label="Entrada de Imagen")
        
        with gr.Row():
            gr.Column(scale=2) 
            btn = gr.Button("Analizar Outfit", variant="primary", scale=2) 
            gr.Column(scale=2) 
        
    output_img = gr.Image(label="Ropa detectada")
    output_label = gr.Textbox(label="Predicción por tipo de prenda")

    btn.click(
        fn=segment_and_classify_by_type,
        inputs=input_img,
        outputs=[output_img, output_label]
    )

if __name__ == "__main__":
    try:
        demo.launch(share=True)
    except Exception as e:
        print(f"Error: {e}")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
c:\Users\andsa\anaconda3\envs\shoeclasificator\lib\site-packages\transformers\image_processing_base.py:417: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'reduce_labels'
  image_processor = cls(**image_processor_dict)


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://92abed60c14c6a0093.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
